In [1]:
import os

In [2]:
def get_file_contents(filename):
    """ Given a filename,
        return the contents of that file
    """
    try:
        with open(filename, 'r') as f:
            # It's assumed our file contains a single line,
            # with our API key
            return f.read().strip()
    except FileNotFoundError:
        print("'%s' file not found" % filename)

In [ ]:
filename = "..\\GoogleAPIKey.txt"

In [4]:
os.environ['GOOGLE_API_KEY'] = get_file_contents(filename)

In [ ]:
filename_grokKey = "..\\GroqAPIKey.txt"

In [6]:
os.environ['GROQ_API_KEY'] = get_file_contents(filename_grokKey)

### Coversation Buffer Memory

In [7]:
from langchain import PromptTemplate
from langchain.chains import LLMChain
from langchain_groq import ChatGroq
from langchain_google_genai import GoogleGenerativeAI
from langchain.memory import ConversationBufferMemory


In [8]:
#gemini_llm = ChatGoogleGenerativeAI(model='gemini-1.5-pro-001')
lamma_llm = ChatGroq(model="llama-3.1-8b-instant")
#google_llm = GoogleGenerativeAI(model='gemini-1.5-pro-001')

In [9]:
google_llm = GoogleGenerativeAI(model='gemini-1.5-pro-001')

In [10]:
prompt_template = PromptTemplate(
    input_variables = ["history","new_input"],
    template="""You are having a chat with human.
        previous chat:{history}
        Human: {new_input}
        Assistant:"""
)

In [12]:
memory_1=ConversationBufferMemory()
memory_2=ConversationBufferMemory()

In [14]:

chain_1 = LLMChain(llm=lamma_llm, prompt=prompt_template, memory=memory_1, verbose=True)

In [15]:
chain_1.invoke({"new_input":"Which country is the biggest exporter of cotton?"})



> Entering new LLMChain chain...
Prompt after formatting:
You are having a chat with human.
        previous chat:
        Human: Which country is the biggest exporter of cotton?
        Assistant:

> Finished chain.


{'new_input': 'Which country is the biggest exporter of cotton?',
 'history': '',
 'text': 'China is currently the largest exporter of cotton, accounting for a significant share of global cotton exports.'}

In [16]:
chain_1.invoke({"new_input":"What is the total value of cotton exports of this country"})



> Entering new LLMChain chain...
Prompt after formatting:
You are having a chat with human.
        previous chat:Human: Which country is the biggest exporter of cotton?
AI: China is currently the largest exporter of cotton, accounting for a significant share of global cotton exports.
        Human: What is the total value of cotton exports of this country
        Assistant:

> Finished chain.


{'new_input': 'What is the total value of cotton exports of this country',
 'history': 'Human: Which country is the biggest exporter of cotton?\nAI: China is currently the largest exporter of cotton, accounting for a significant share of global cotton exports.',
 'text': "Unfortunately, I don't have the most up-to-date information on the exact total value of China's cotton exports. However, I can provide you with some general information on China's cotton exports. \n\nChina's cotton exports have fluctuated in recent years, and the total value can vary depending on factors such as global demand, prices, and trade policies. According to the Observatory for Economic Complexity (OEC), China's cotton exports have been steadily increasing over the years, with the total value reaching around $6.5 billion in 2020. However, this information may not reflect the current market situation.\n\nIf you're looking for the most recent data, I would recommend checking the official statistics from the Chi

In [17]:
chain_2 = LLMChain(llm=google_llm, prompt=prompt_template, memory=memory_2, verbose=True)

In [18]:
chain_2.invoke({"new_input":"Which country is the biggest exporter of cotton?"})



> Entering new LLMChain chain...
Prompt after formatting:
You are having a chat with human.
        previous chat:
        Human: Which country is the biggest exporter of cotton?
        Assistant:

> Finished chain.


{'new_input': 'Which country is the biggest exporter of cotton?',
 'history': '',
 'text': 'India is the biggest exporter of cotton as of 2021. '}

In [19]:
chain_2.invoke({"new_input":"What is the total value of cotton exports of this country"})



> Entering new LLMChain chain...
Prompt after formatting:
You are having a chat with human.
        previous chat:Human: Which country is the biggest exporter of cotton?
AI: India is the biggest exporter of cotton as of 2021. 
        Human: What is the total value of cotton exports of this country
        Assistant:

> Finished chain.


{'new_input': 'What is the total value of cotton exports of this country',
 'history': 'Human: Which country is the biggest exporter of cotton?\nAI: India is the biggest exporter of cotton as of 2021. ',
 'text': 'I need more information to answer your question.  You asked about "this country," but in your previous message, you didn\'t specify a country, you just asked about the biggest cotton exporter. \n\nTo give you the total value of cotton exports, I need to know:\n\n*  **Which country are you interested in?**  Do you mean India (the biggest exporter) or another country? \n\nPlease tell me the specific country you\'re asking about! '}

In [20]:
from langchain_core.messages import SystemMessage
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder
)

In [21]:
chatPrompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content="You are having a chat with the human."
        ),
        MessagesPlaceholder(
            variable_name="history"
        ),
        HumanMessagePromptTemplate.from_template(
            "{new_input}"
        ),
    ]
)

In [22]:
chat_memory = ConversationBufferMemory(return_messages=True)

In [23]:
chat_lamma_llm = ChatGroq(model="llama-3.1-8b-instant")

In [24]:
chat_chain = LLMChain(llm=chat_lamma_llm,prompt=chatPrompt,memory=chat_memory,verbose=True) 

In [25]:
chat_chain.invoke({"new_input":"Which country is the biggest exporter of cotton?"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are having a chat with the human.
Human: Which country is the biggest exporter of cotton?

> Finished chain.


{'new_input': 'Which country is the biggest exporter of cotton?',
 'history': [HumanMessage(content='Which country is the biggest exporter of cotton?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="China is the largest exporter of cotton, accounting for approximately 30-40% of the world's total cotton exports.", additional_kwargs={}, response_metadata={})],
 'text': "China is the largest exporter of cotton, accounting for approximately 30-40% of the world's total cotton exports."}

In [ ]:
chat_chain.invoke({"new_input":"What is the total value of cotton exports of this country?"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are having a chat with the human.
Human: Which country is the biggest exporter of cotton?
AI: China is the largest exporter of cotton, accounting for approximately 30-40% of the world's total cotton exports.
Human: What is the total value of cotton exports of this country?

> Finished chain.


{'new_input': 'What is the total value of cotton exports of this country?',
 'history': [HumanMessage(content='Which country is the biggest exporter of cotton?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="China is the largest exporter of cotton, accounting for approximately 30-40% of the world's total cotton exports.", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is the total value of cotton exports of this country?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="The total value of cotton exports for China is difficult to estimate exactly, as it can vary greatly from year to year due to market fluctuations and other factors. However, based on recent data, in 2020-2021, China's cotton exports were valued at approximately $3.4 billion USD. \n\nHowever, please note that the value can be affected by factors like the global market, international trade policies, and other economic factors.", additional_kwargs={}, respons

### Changing the memory variable name

In [27]:
chatPrompt_2 = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content="You are having a chat with the human."
        ),
        MessagesPlaceholder(
            variable_name="prev_conv"
        ),
        HumanMessagePromptTemplate.from_template(
            "{new_input}"
        ),
    ]
)

In [28]:
chat_memory_2 = ConversationBufferMemory(memory_key="prev_conv",return_messages=True)

In [29]:
chat_chain_2 = LLMChain(llm=chat_lamma_llm,prompt=chatPrompt_2,memory=chat_memory_2,verbose=True) 

In [30]:
chat_chain_2.invoke({"new_input":"Which country is the biggest exporter of cotton?"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are having a chat with the human.
Human: Which country is the biggest exporter of cotton?

> Finished chain.


{'new_input': 'Which country is the biggest exporter of cotton?',
 'prev_conv': [HumanMessage(content='Which country is the biggest exporter of cotton?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="The biggest exporter of cotton is China, accounting for a significant portion of the world's cotton exports.", additional_kwargs={}, response_metadata={})],
 'text': "The biggest exporter of cotton is China, accounting for a significant portion of the world's cotton exports."}

### Add messages to memory

In [31]:
sample_memory = ConversationBufferMemory(memory_key="history")

In [34]:
sample_memory.load_memory_variables({})

{'history': ''}

In [35]:
sample_memory.chat_memory.add_user_message("Which country is the biggest exporter of cotton?")
sample_memory.chat_memory.add_ai_message("China")

In [36]:
sample_memory.load_memory_variables({})

{'history': 'Human: Which country is the biggest exporter of cotton?\nAI: China'}

In [37]:
prompt_template_2 = PromptTemplate(
    input_variables = ["history","new_input"],
    template="""You are having a chat with human.
        previous chat:{history}
        Human: {new_input}
        Assistant:"""
)

In [38]:
chain_3 = LLMChain(llm=lamma_llm,prompt=prompt_template,memory=sample_memory,verbose=True)

In [39]:
chain_3.invoke({"new_input":"What is the total value of cotton exports of this country?"})



> Entering new LLMChain chain...
Prompt after formatting:
You are having a chat with human.
        previous chat:Human: Which country is the biggest exporter of cotton?
AI: China
        Human: What is the total value of cotton exports of this country?
        Assistant:

> Finished chain.


{'new_input': 'What is the total value of cotton exports of this country?',
 'history': 'Human: Which country is the biggest exporter of cotton?\nAI: China',
 'text': "However, I need a bit more information to provide an accurate answer. The total value of cotton exports from China can vary from year to year, influenced by factors such as global demand, production costs, and market trends.\n\nThat being said, according to the United Nations Comtrade database, in 2020, China's total value of cotton exports was approximately $12.4 billion USD. However, I would like to clarify that this information might not reflect the current or more recent data.\n\nIf you need the most up-to-date information or a specific year's data, please let me know, and I'll do my best to provide you with the accurate information."}

### Conversation Chain instead of LLMChain

In [41]:
from langchain.chains import ConversationChain

In [42]:
memory_3 = ConversationBufferMemory(memory_key="history")

In [45]:
chain_4 = ConversationChain(llm=lamma_llm,memory=memory_3,verbose=True)

In [46]:
chain_4.invoke({"input":"Which country is the biggest exporter of cotton?"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Which country is the biggest exporter of cotton?
AI:

> Finished chain.


{'input': 'Which country is the biggest exporter of cotton?',
 'history': '',
 'response': "China is the world's largest producer of cotton, accounting for approximately 33% of global production. However, when it comes to cotton exports, the country that tops the list is actually the United States. According to the International Trade Centre, in 2020, the United States was the largest exporter of cotton, with a total export value of around $5.6 billion.\n\nChina does export cotton, but its domestic market and production volumes are so large that it often uses its cotton for internal consumption rather than for export. India, Pakistan, and Brazil are also significant cotton-exporting countries, but the US takes the lead in terms of global cotton exports."}

In [47]:
print(chain_4.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [48]:
chain_4.invoke({"input":"What is the total value of cotton exports of this country?"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Which country is the biggest exporter of cotton?
AI: China is the world's largest producer of cotton, accounting for approximately 33% of global production. However, when it comes to cotton exports, the country that tops the list is actually the United States. According to the International Trade Centre, in 2020, the United States was the largest exporter of cotton, with a total export value of around $5.6 billion.

China does export cotton, but its domestic market and production volumes are so large that it often uses its cotton for internal consumption rather than for export. India, Pakistan, and Brazil are also significant cotton-exporting countries, but the 

{'input': 'What is the total value of cotton exports of this country?',
 'history': "Human: Which country is the biggest exporter of cotton?\nAI: China is the world's largest producer of cotton, accounting for approximately 33% of global production. However, when it comes to cotton exports, the country that tops the list is actually the United States. According to the International Trade Centre, in 2020, the United States was the largest exporter of cotton, with a total export value of around $5.6 billion.\n\nChina does export cotton, but its domestic market and production volumes are so large that it often uses its cotton for internal consumption rather than for export. India, Pakistan, and Brazil are also significant cotton-exporting countries, but the US takes the lead in terms of global cotton exports.",
 'response': "According to the International Trade Centre, the total value of cotton exports of the United States in 2020 was around $5.6 billion. However, I can also provide you w

In [49]:
print(chain_4.memory.buffer)

Human: Which country is the biggest exporter of cotton?
AI: China is the world's largest producer of cotton, accounting for approximately 33% of global production. However, when it comes to cotton exports, the country that tops the list is actually the United States. According to the International Trade Centre, in 2020, the United States was the largest exporter of cotton, with a total export value of around $5.6 billion.

China does export cotton, but its domestic market and production volumes are so large that it often uses its cotton for internal consumption rather than for export. India, Pakistan, and Brazil are also significant cotton-exporting countries, but the US takes the lead in terms of global cotton exports.
Human: What is the total value of cotton exports of this country?
AI: According to the International Trade Centre, the total value of cotton exports of the United States in 2020 was around $5.6 billion. However, I can also provide you with more detailed information on c